# Transformation N°2 : calcul des délais d'émission de la balise et des délai de réception du serveur

In [5]:
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
import  calendar as cal, datetime as dt

In [7]:
df = pd.read_csv("DATA/PREPADATA/positionshistoriqueFiltreTrieeUnique.csv",sep =';',header = 0 , parse_dates=[0,1])

In [8]:
maliste=['10002', '10004', '10006', '10008', '10009', '10010', '10011', '11002', '11003', '11004', '11005', '11007', '11008', '12001', '12002', '12003', '12004', '12005', '12006', '12007', '12008', '12009', '12010']
for k in maliste :
    df[df['idBalise']==int(k)].to_csv("DATA/TRANSFORMATION2/positionshistorique_"+k+".csv", sep = ';',index = False)
    

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165992 entries, 0 to 165991
Data columns (total 13 columns):
DateReception_HeureReception    165992 non-null datetime64[ns]
DateGPS_HeureGPS                165992 non-null datetime64[ns]
idBalise                        165992 non-null int64
Longitude                       165992 non-null float64
Latitude                        165992 non-null float64
Vitesse                         165992 non-null int64
Direction                       165992 non-null int64
Altitude                        165992 non-null int64
Satellites                      165992 non-null int64
MessageId                       165992 non-null int64
Puissance                       165992 non-null float64
VoltageBatterie                 165992 non-null float64
Distance                        165992 non-null int64
dtypes: datetime64[ns](2), float64(4), int64(7)
memory usage: 16.5 MB


In [10]:
maliste = ['10002', '10004', '10006', '10008', '10009', '10010', '10011', '11002', '11003', '11004', '11005', '11007', '11008', '12001', '12002', '12003', '12004', '12005', '12006', '12007', '12008', '12009', '12010']
for k in maliste :
    dfk1 = pd.read_csv("DATA/TRANSFORMATION2/positionshistorique_"+k+".csv",sep =';',header = 0 , parse_dates=[0,1])
    dfk2 = pd.read_csv("DATA/TRANSFORMATION2/positionshistorique_"+k+".csv",sep =';',header = 0 , parse_dates=[0,1])
    dfk1L1 =df.head(1)
    dfk2 = pd.concat([dfk1L1, dfk2], axis=0, join='outer', ignore_index=True)
    dfk2.columns = [x+"Prec" for x in df.columns]
    resultk = pd.concat([dfk1, dfk2], axis=1, join='inner')
    resultk.drop(['idBalisePrec', 'LongitudePrec', 'LatitudePrec','VitessePrec', 'DirectionPrec', 'AltitudePrec', 'SatellitesPrec','MessageIdPrec', 'PuissancePrec', 'VoltageBatteriePrec'], axis = 1, inplace = True) 
    resultk.drop([0], axis = 0, inplace = True) 
    resultk.to_csv("DATA/TRANSFORMATION2/positionshistoriqueW_"+k+".csv", sep = ';',index = False)
    

# Normalisation et ajout des délais à étudier

In [11]:
maliste=['10002', '10004', '10006', '10008', '10009', '10010', '10011', '11002', '11003', '11004', '11005', '11007', '11008', '12001', '12002', '12003', '12004', '12005', '12006', '12007', '12008', '12009', '12010']
for k in maliste :
    data = pd.read_csv("DATA/TRANSFORMATION2/positionshistoriqueW_"+k+".csv",sep =';',header = 0 , parse_dates=[0,1,13,14])
    data["DelaiServeur"] =data.apply ( lambda row : ( int((row["DateReception_HeureReception"]-row["DateReception_HeureReceptionPrec"]).total_seconds() ) )  , axis = 1 )
    data["DelaiBalise"] =data.apply ( lambda row : ( int((row["DateGPS_HeureGPS"]-row["DateGPS_HeureGPSPrec"]).total_seconds() ) )  , axis = 1 )
    data["DeltaDist"] = data.apply ( lambda row : ( int(row["Distance"]-int(float(row["DistancePrec"])))), axis = 1 )
    data["Lng"] = data.apply ( lambda row : ( round(row["Longitude"],5)), axis = 1 )
    data["Lat"] = data.apply ( lambda row : ( round(row["Latitude"],5)), axis = 1 )
    data["Power"] = data.apply ( lambda row : ( round(row["Puissance"],2)), axis = 1 )
    data["Volt"] = data.apply ( lambda row : ( round(row["VoltageBatterie"],2)), axis = 1 )
    data.drop(['Longitude', 'Latitude', 'Puissance','VoltageBatterie'], axis = 1, inplace = True) 
    data.to_csv("DATA/TRANSFORMATION2/a_positionshistorique_"+k+".csv", sep = ';',index = False)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6946 entries, 0 to 6945
Data columns (total 19 columns):
DateReception_HeureReception        6946 non-null datetime64[ns]
DateGPS_HeureGPS                    6946 non-null datetime64[ns]
idBalise                            6946 non-null int64
Vitesse                             6946 non-null int64
Direction                           6946 non-null int64
Altitude                            6946 non-null int64
Satellites                          6946 non-null int64
MessageId                           6946 non-null int64
Distance                            6946 non-null int64
DateReception_HeureReceptionPrec    6946 non-null datetime64[ns]
DateGPS_HeureGPSPrec                6946 non-null datetime64[ns]
DistancePrec                        6946 non-null int64
DelaiServeur                        6946 non-null int64
DelaiBalise                         6946 non-null int64
DeltaDist                           6946 non-null int64
Lng              

In [13]:
data.shape

(6946, 19)

In [14]:
df10002 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_10002.csv",sep =';',header = 0 , parse_dates=[1,2])
df10004 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_10004.csv",sep =';',header = 0 , parse_dates=[1,2])
df10006 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_10006.csv",sep =';',header = 0 , parse_dates=[1,2])
df10008 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_10008.csv",sep =';',header = 0 , parse_dates=[1,2])
df10009 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_10009.csv",sep =';',header = 0 , parse_dates=[1,2])
df10010 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_10010.csv",sep =';',header = 0 , parse_dates=[1,2])
df10011 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_10011.csv",sep =';',header = 0 , parse_dates=[1,2])

In [15]:
df11002 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_11002.csv",sep =';',header = 0 , parse_dates=[1,2])
df11003 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_11003.csv",sep =';',header = 0 , parse_dates=[1,2])
df11004 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_11004.csv",sep =';',header = 0 , parse_dates=[1,2])
df11005 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_11005.csv",sep =';',header = 0 , parse_dates=[1,2])
df11007 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_11007.csv",sep =';',header = 0 , parse_dates=[1,2])
df11008 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_11008.csv",sep =';',header = 0 , parse_dates=[1,2])


In [18]:
df12001 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_12001.csv",sep =';',header = 0 , parse_dates=[1,2])
df12002 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_12002.csv",sep =';',header = 0 , parse_dates=[1,2])
df12003 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_12003.csv",sep =';',header = 0 , parse_dates=[1,2])
df12004 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_12004.csv",sep =';',header = 0 , parse_dates=[1,2])
df12005 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_12005.csv",sep =';',header = 0 , parse_dates=[1,2])
df12006 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_12006.csv",sep =';',header = 0 , parse_dates=[1,2])
df12007 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_12007.csv",sep =';',header = 0 , parse_dates=[1,2])
df12008 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_12008.csv",sep =';',header = 0 , parse_dates=[1,2])
df12009 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_12009.csv",sep =';',header = 0 , parse_dates=[1,2])
df12010 = pd.read_csv("DATA/TRANSFORMATION2/a_positionshistorique_12010.csv",sep =';',header = 0 , parse_dates=[1,2])

#'12001', '12002', '12003', '12004', '12005', '12006', '12007', '12008', '12009', '12010'

In [19]:
frames10 = [df10002 ,df10004, df10006,df10008 ,df10009, df10010, df10011 ]
result10 = pd.concat(frames10)
result10.to_csv("DATA/TRANSFORMATION2/positionshistorique_10.csv", sep = ';',index = False)

In [20]:
frames11 = [df11002, df11003, df11004, df11005, df11007, df11008]
result11 = pd.concat(frames11)
result11.to_csv("DATA/TRANSFORMATION2/positionshistorique_11.csv", sep = ';',index = False)

In [21]:
frames12 = [df12001, df12002, df12003, df12004, df12005, df12006, df12007, df12008, df12009, df12010 ]
result12 = pd.concat(frames12)
result12.to_csv("DATA/TRANSFORMATION2/positionshistorique_12.csv", sep = ';',index = False)

In [22]:
df10 = pd.read_csv("DATA/TRANSFORMATION2/positionshistorique_10.csv",sep =';',header = 0 , parse_dates=[1,2])
df11 = pd.read_csv("DATA/TRANSFORMATION2/positionshistorique_11.csv",sep =';',header = 0 , parse_dates=[1,2])
df12 = pd.read_csv("DATA/TRANSFORMATION2/positionshistorique_12.csv",sep =';',header = 0 , parse_dates=[1,2])

In [23]:
df10.shape

(52079, 19)

In [24]:
df11.shape

(43361, 19)

In [25]:
df12.shape

(70529, 19)

In [26]:
framesFinal = [df10, df11, df12]
resultFinal = pd.concat(framesFinal)

In [27]:
resultFinal.to_csv("DATA/positionshistoriqueEtude.csv", sep = ';',index = False)

In [36]:
df = pd.read_csv("DATA/positionshistoriqueEtude.csv",sep =';',header = 0 , parse_dates=[0,1,9,10])

In [32]:
#df["DelaiGPSPrec"] = df.apply ( lambda row : int(( row["DateHeureGPS"]-row["DateHeureGPSPrec"]).total_seconds()), axis = 1 )
#df["DelaiReceptionPrec"] = df.apply ( lambda row : int(( row["DateHeureReception"]-row["DateHeureReceptionPrec"]).total_seconds()), axis = 1 )
df["BaliseLabel"] = df.apply ( lambda row : ( "B"+str(row["idBalise"])), axis = 1 )
df["MessageIdLabel"] = df.apply ( lambda row : ( "B"+str(row["MessageId"])), axis = 1 )

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165969 entries, 0 to 165968
Data columns (total 21 columns):
dateHeureRec            165969 non-null datetime64[ns]
dateHeureGPS            165969 non-null datetime64[ns]
idBalise                165969 non-null int64
vitesse                 165969 non-null int64
direction               165969 non-null int64
altitude                165969 non-null int64
satellites              165969 non-null int64
messageId               165969 non-null int64
distance                165969 non-null int64
dateHeureServeurPrec    165969 non-null datetime64[ns]
dateHeureBalisePrec     165969 non-null datetime64[ns]
distancePrec            165969 non-null int64
delaiServeur            165969 non-null int64
delaiBalise             165969 non-null int64
deltaDistance           165969 non-null int64
lng                     165969 non-null float64
lat                     165969 non-null float64
power                   165969 non-null float64
volt               

In [38]:
df.columns = ['dateHeureRec', 'dateHeureGPS', 'idBalise',
       'vitesse', 'direction', 'altitude', 'satellites', 'messageId',
       'distance', 'dateHeureServeurPrec',
       'dateHeureBalisePrec', 'distancePrec', 'delaiServeur', 'delaiBalise', 'deltaDistance',
       'lng', 'lat', 'power', 'volt','labelBalise','labelMessage']


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165969 entries, 0 to 165968
Data columns (total 21 columns):
dateHeureRec            165969 non-null datetime64[ns]
dateHeureGPS            165969 non-null datetime64[ns]
idBalise                165969 non-null int64
vitesse                 165969 non-null int64
direction               165969 non-null int64
altitude                165969 non-null int64
satellites              165969 non-null int64
messageId               165969 non-null int64
distance                165969 non-null int64
dateHeureServeurPrec    165969 non-null datetime64[ns]
dateHeureBalisePrec     165969 non-null datetime64[ns]
distancePrec            165969 non-null int64
delaiServeur            165969 non-null int64
delaiBalise             165969 non-null int64
deltaDistance           165969 non-null int64
lng                     165969 non-null float64
lat                     165969 non-null float64
power                   165969 non-null float64
volt               

# Sauvegarde du fichier à etudier

In [41]:
df.to_csv("DATA/positionshistoriqueEtude.csv", sep = ';',index = False)